In [1]:
# Installs geemap package
import subprocess
import os
import pandas as pd
import geopandas as gpd
import json #do I need this if I'm using geopandas?
import rasterio as rst

import geemap
import ee

In [2]:
import ee
import geemap

In [3]:
print(os.getcwd())
#print(os.listdir('.'))

/home/jennaguffogg/workspace/sensand/repos/jenna-test


In [4]:
ee.Authenticate()
ee.Initialize()

geom_gpd = gpd.read_file('marysville-fire-forest-loss-test-area.geojson')
geom = geemap.geopandas_to_ee(geom_gpd)


gio: https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6mS9zoulirZYPIyKhHFefEE5ViFcpMfq8lj67Lm8JUw&tc=NI-Kft-jH1KojatM-fmW9dYgJNJXCMvI6H4f9RzHJV8&cc=2e4xDTZNmMrsjCVSc88r4q64y1FNMho9wL84zlSWczM: Operation not supported



Successfully saved authorization token.


In [5]:
# Extract DW data for 2 target years, 2018 and 2022
# use label band because its the index of the band with the highest estimated probability
dw_trees_col_2018 = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(geom).filterDate('2018-01-01','2019-01-01').select('label').median() #can't join as this is median reduced
dw_trees_col_2022 = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(geom).filterDate('2022-01-01','2023-01-01').select('label').median()
dw_trees_image_2018 = dw_trees_col_2018.lt(2).And(dw_trees_col_2018.gt(0)).clip(geom) #the code for 'trees' is 1, so we isolate pixels with 1
dw_trees_image_2022 = dw_trees_col_2022.lt(2).And(dw_trees_col_2022.gt(0)).clip(geom)

#combine them to find spots that were trees in 2018 and aren't now
dw_trees_image_2018_2022_diff = dw_trees_col_2018.lt(2).And(dw_trees_col_2022.gt(2)).clip(geom)

In [6]:
#sentinel2 cloud masking and making 2022 year composite
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
  # can remove the divide, but then set vis params to max:3000 instead of max:0.3
  return image.updateMask(mask).divide(10000) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])
# }

#if your sent-2 image looks faded, check you're using the harmonised collection'
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2022-01-01', '2022-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds)

composite = collection.median().clip(geom)


In [7]:
#initialise map and add visualisation settings for the layers
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
change_vis_params = {'min':0, 'max':1, 'palette':['white','red']}
m2 = geemap.Map()
m2.centerObject(geom)


m2.addLayer(dw_trees_image_2018.updateMask(dw_trees_image_2018), change_vis_params, 'dynamic world trees 2018', shown=False)
m2.addLayer(dw_trees_image_2022.updateMask(dw_trees_image_2022), change_vis_params, 'dynamic world trees 2022', shown=False)
m2.addLayer(composite, vis_params, 'RGB')
m2.addLayer(dw_trees_image_2018_2022_diff.updateMask(dw_trees_image_2018_2022_diff), change_vis_params, 'dynamic world tree loss 2018-2022')

#m2.addLayer(geom, {})
m2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

At the moment te DW data is only for 2018 and 2022, not inbetween. This can be adjusted so that DW data after 2018 is used to find all areas of tree loss, rather than just highlighting the 2022 logging coupes, which is what is happening ATM.


In [8]:
# Save Dynamic World class data in GeoTIFF format
#output_path = 'landcover.tif'
#landcover = geemap.dynamic_world(region, start_date, end_date, return_type='class')
#geemap.ee_export_image(landcover, filename=output_path, scale=10, region=region, file_per_band=False)